In [2]:
import pandas as pd
import numpy as np
from scipy.io import arff
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
import functools
import warnings
import joblib
import time
from collections import defaultdict
from scipy.optimize import curve_fit

# Suppress warnings
warnings.filterwarnings("ignore", category=UserWarning, module='sklearn')
optuna.logging.set_verbosity(optuna.logging.WARNING)

# --- 1. RandomForestMultiFidelityPruner (Unchanged) ---
class RandomForestMultiFidelityPruner(optuna.pruners.BasePruner):
    def __init__(self, 
                 min_intermediate_steps=3, 
                 pruning_quantile=0.5, 
                 grace_period_steps=2, 
                 predictive_model_type='saturating_curve', 
                 debug_mode=True):
        self.min_intermediate_steps = min_intermediate_steps
        self.pruning_quantile = pruning_quantile
        self.grace_period_steps = grace_period_steps
        self.predictive_model_type = predictive_model_type
        self.trial_learning_curves = defaultdict(list)
        self.debug_mode = debug_mode
        self.predictions_log = []

    def prune(self, study, trial):
        trial_id = trial.number
        intermediate_values = trial.intermediate_values
        
        if self.debug_mode:
            print(f"\n=== DEBUGGING TRIAL {trial_id} ===")
            print(f"Intermediate values: {intermediate_values}")
        
        if not intermediate_values:
            if self.debug_mode:
                print(f"[Debug Trial {trial_id}]: No intermediate values reported")
            return False
        
        current_step = max(intermediate_values.keys())
        current_value = intermediate_values[current_step]
        
        if self.debug_mode:
            print(f"Current step: {current_step}, Current value: {current_value}")
        
        if trial_id not in self.trial_learning_curves:
            self.trial_learning_curves[trial_id] = []
        if not self.trial_learning_curves[trial_id] or \
           self.trial_learning_curves[trial_id][-1][0] != current_step:
            self.trial_learning_curves[trial_id].append((current_step, current_value))
        
        if len(intermediate_values) < self.min_intermediate_steps:
            if self.debug_mode:
                print(f"[Debug Trial {trial_id}]: Not enough steps ({len(intermediate_values)} < {self.min_intermediate_steps})")
            return False
        
        if current_step < self.grace_period_steps:
            if self.debug_mode:
                print(f"[Debug Trial {trial_id}]: In grace period (step {current_step} < {self.grace_period_steps})")
            return False
        
        steps = np.array(list(intermediate_values.keys()))
        values = np.array(list(intermediate_values.values()))
        
        if self.debug_mode:
            print(f"Steps for prediction: {steps}")
            print(f"Values for prediction: {values}")
        
        try:
            predicted_final_performance = self._predict_final_performance(
                steps, values, trial.params.get('n_estimators', 250), trial_id)
            
            if self.debug_mode:
                print(f"Predicted final performance: {predicted_final_performance}")
                
        except Exception as e:
            if self.debug_mode:
                print(f"[Debug Trial {trial_id}]: Prediction failed: {e}")
            return False
        
        completed_trial_values = [
            t.value for t in study.trials 
            if t.state == optuna.trial.TrialState.COMPLETE and t.value is not None
        ]
        
        if not completed_trial_values:
            if self.debug_mode:
                print(f"[Debug Trial {trial_id}]: No completed trials yet")
            return False
        
        threshold = np.quantile(completed_trial_values, self.pruning_quantile)
        
        if self.debug_mode:
            print(f"Completed trial values: {completed_trial_values}")
            print(f"Threshold (quantile {self.pruning_quantile}): {threshold}")
            print(f"Prediction vs threshold: {predicted_final_performance:.4f} vs {threshold:.4f}")
        
        should_prune = predicted_final_performance < threshold
        
        if self.debug_mode:
            decision = "PRUNE" if should_prune else "CONTINUE"
            print(f"Decision: {decision}")
            if should_prune:
                print(f"🔥 PRUNING: {predicted_final_performance:.4f} < {threshold:.4f}")
            else:
                print(f"✅ CONTINUING: {predicted_final_performance:.4f} >= {threshold:.4f}")
        
        self.predictions_log.append({
            'trial_id': trial_id,
            'step': current_step,
            'predicted': predicted_final_performance,
            'threshold': threshold,
            'pruned': should_prune,
            'intermediate_values': dict(intermediate_values)
        })
        
        return should_prune

    def _predict_final_performance(self, steps, values, max_n_estimators, trial_id):
        if self.predictive_model_type == 'saturating_curve':
            return self._fit_saturating_curve(steps, values, trial_id)
        elif self.predictive_model_type == 'linear_extrapolation':
            return self._linear_extrapolation(steps, values)
        else:
            return values[-1]
    
    def _fit_saturating_curve(self, steps, values, trial_id):
        def saturating_func(x, a, b, c):
            return a - b * np.exp(-c * x)
        
        try:
            p0 = [max(values), max(values) - min(values), 0.1]
            popt, _ = curve_fit(saturating_func, steps, values, p0=p0, maxfev=1000)
            final_step = max(steps) * 3
            predicted = saturating_func(final_step, *popt)
            
            if self.debug_mode:
                print(f"[Debug Trial {trial_id}]: Curve fit params: a={popt[0]:.4f}, b={popt[1]:.4f}, c={popt[2]:.4f}")
                print(f"[Debug Trial {trial_id}]: Extrapolated to step {final_step}: {predicted:.4f}")
            
            return predicted
            
        except Exception as e:
            if self.debug_mode:
                print(f"[Debug Trial {trial_id}]: Curve fitting failed: {e}, using linear extrapolation")
            return self._linear_extrapolation(steps, values)
    
    def _linear_extrapolation(self, steps, values):
        if len(steps) < 2:
            return values[-1]
        
        slope = (values[-1] - values[-2]) / (steps[-1] - steps[-2])
        target_step = steps[-1] * 2
        predicted = values[-1] + slope * (target_step - steps[-1])
        
        return predicted
    
    def get_prediction_analysis(self):
        return self.predictions_log

# --- 2. Dataset Loading and Preprocessing ---
def load_and_preprocess_arff(filepath, target_column_name=None):
    data, meta = arff.loadarff(filepath)
    df = pd.DataFrame(data)

    for col in df.columns:
        if df[col].dtype == 'object':
            try:
                df[col] = df[col].apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
            except (UnicodeDecodeError, AttributeError):
                pass

    if target_column_name is None:
        target_column_name = df.columns[-1]

    X = df.drop(columns=[target_column_name])
    y = df[target_column_name]

    categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()
    numerical_features = X.select_dtypes(include=np.number).columns.tolist()

    feature_names = X.columns.tolist()
    categorical_features_idx = [feature_names.index(col) for col in categorical_features]

    print(f"Loaded {filepath}. Shape: {df.shape}")
    print(f"Target column: '{target_column_name}'")
    print(f"Numerical features: {numerical_features}")
    print(f"Categorical features detected: {categorical_features}")

    return X, y, feature_names, categorical_features_idx

def get_preprocessor(numerical_features, categorical_features):
    from sklearn.preprocessing import OneHotEncoder
    transformers = []
    if numerical_features:
        transformers.append(('num', 'passthrough', numerical_features))
    if categorical_features:
        transformers.append(('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features))

    preprocessor = ColumnTransformer(
        transformers=transformers,
        remainder='drop'
    )
    return preprocessor

# --- 3. Corrected Objective Function ---
def objective(trial, X_train_raw, y_train_encoded, X_val_raw, y_val_encoded,
              task_type='classification', preprocessor=None, max_n_estimators_upper_bound=250,
              debug_mode=True):  # Added debug_mode parameter
    n_estimators = trial.suggest_int("n_estimators", 50, max_n_estimators_upper_bound)
    max_depth = trial.suggest_int("max_depth", 2, 32)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 20)
    max_features_val = trial.suggest_categorical("max_features", ['sqrt', 'log2', 0.5, 0.7, 1.0])
    criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])
    class_weight = trial.suggest_categorical("class_weight", [None, "balanced"])

    rf_model = RandomForestClassifier(
        n_estimators=1,
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features_val,
        criterion=criterion,
        class_weight=class_weight,
        oob_score=True,
        random_state=42,
        warm_start=True,
        n_jobs=-1
    )

    if preprocessor:
        X_train_processed = preprocessor.fit_transform(X_train_raw)
        X_val_processed = preprocessor.transform(X_val_raw)
    else:
        X_train_processed = X_train_raw.to_numpy()
        X_val_processed = X_val_raw.to_numpy()

    fidelity_schedule = [0.02, 0.05, 0.1, 0.2, 0.5, 1.0]
    fidelity_steps = sorted(list(set([max(1, int(n_estimators * p)) for p in fidelity_schedule])))

    for step, current_n_estimators in enumerate(fidelity_steps):
        if current_n_estimators > rf_model.n_estimators:
            rf_model.n_estimators = current_n_estimators
            rf_model.fit(X_train_processed, y_train_encoded)

        if rf_model.n_estimators > 1 and hasattr(rf_model, 'oob_score_'):
            current_oob_score = rf_model.oob_score_
        else:
            current_oob_score = np.nan

        trial.report(current_oob_score, step)

        if debug_mode:  # Fixed: Use debug_mode parameter instead of self.debug_mode
            print(f"Trial {trial.number}, Step {step}: {current_n_estimators} trees, OOB: {current_oob_score:.4f}")

        if trial.should_prune():
            print(f"Trial {trial.number} pruned at step {step} ({current_n_estimators} estimators, OOB: {current_oob_score:.4f})")
            raise optuna.TrialPruned()

    y_pred = rf_model.predict(X_val_processed)

    if len(np.unique(y_val_encoded)) == 2:
        y_proba = rf_model.predict_proba(X_val_processed)[:, 1]
        final_metric = roc_auc_score(y_val_encoded, y_proba)
        metric_name = "AUC"
    else:
        final_metric = accuracy_score(y_val_encoded, y_pred)
        metric_name = "Accuracy"

    print(f"Trial {trial.number} completed with {metric_name}: {final_metric:.4f}")
    return final_metric

# --- 4. Running the Optuna Study ---
N_TRIALS = 100
RANDOM_STATE = 42
DATASET_PATH = 'datasets/php0iVrYT.arff'
TARGET_COLUMN = 'Class'
TASK_TYPE = 'classification'

# Load and preprocess data
X_raw, y_raw, feature_names, categorical_features_idx = load_and_preprocess_arff(
    DATASET_PATH, target_column_name=TARGET_COLUMN
)

le = LabelEncoder()
y_encoded = le.fit_transform(y_raw)
print(f"Encoded target classes: {le.classes_}")

numerical_feats = X_raw.columns.tolist()
categorical_feats = []

data_preprocessor = get_preprocessor(numerical_feats, categorical_feats)

X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_raw, y_encoded, test_size=0.2, random_state=RANDOM_STATE,
    stratify=y_encoded
)

# Initialize pruner
custom_pruner = RandomForestMultiFidelityPruner(
    min_intermediate_steps=3,
    pruning_quantile=0.5,
    grace_period_steps=2,
    predictive_model_type='saturating_curve',
    debug_mode=True
)

# Create Optuna study
study = optuna.create_study(
    direction="maximize",
    pruner=custom_pruner,
    sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE),
    study_name="blood_transfusion_rf_hpo_study_fixed_pruner",
    storage="sqlite:///blood_transfusion_rf_hpo_fixed.db",
    load_if_exists=True
)

objective_with_args = functools.partial(
    objective,
    X_train_raw=X_train_split,
    y_train_encoded=y_train_split,
    X_val_raw=X_val_split,
    y_val_encoded=y_val_split,
    task_type=TASK_TYPE,
    preprocessor=data_preprocessor,
    max_n_estimators_upper_bound=250,
    debug_mode=True  # Pass debug_mode to objective
)

print(f"Starting Optuna study with {N_TRIALS} trials for Blood Transfusion dataset (FIXED PRUNER)...")
study.optimize(objective_with_args, n_trials=N_TRIALS, show_progress_bar=True)

# Print results
print("\nOptimization finished.")
print(f"Best trial value (AUC/Accuracy): {study.best_trial.value:.4f}")
print("Best hyperparameters:")
for key, value in study.best_trial.params.items():
    print(f"  {key}: {value}")

pruned_trials = study.get_trials(deepcopy=False, states=[optuna.trial.TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[optuna.trial.TrialState.COMPLETE])
print(f"Number of pruned trials: {len(pruned_trials)}")
print(f"Number of complete trials: {len(complete_trials)}")

# Analyze pruning predictions
print("\nPruning prediction analysis:")
for log in custom_pruner.get_prediction_analysis():
    print(f"Trial {log['trial_id']}, Step {log['step']}: Predicted {log['predicted']:.4f}, Threshold {log['threshold']:.4f}, Pruned: {log['pruned']}")

# --- 5. Final Model Evaluation ---
def evaluate_final_model(best_params, X_train_full, y_train_full, X_test_full, y_test_full,
                         task_type='classification', preprocessor=None):
    start_time = time.time()

    if preprocessor:
        X_train_processed = preprocessor.fit_transform(X_train_full)
        X_test_processed = preprocessor.transform(X_test_full)
    else:
        X_train_processed = X_train_full.to_numpy()
        X_test_processed = X_test_full.to_numpy()

    model = RandomForestClassifier(random_state=42, n_jobs=-1, **best_params)

    model.fit(X_train_processed, y_train_full)
    fit_time = time.time() - start_time

    y_pred = model.predict(X_test_processed)

    if task_type == 'classification':
        if len(np.unique(y_test_full)) == 2:
            y_proba = model.predict_proba(X_test_processed)[:, 1]
            auc_score = roc_auc_score(y_test_full, y_proba)
            print(f"Final Model AUC: {auc_score:.4f}")
            final_metric = auc_score
            metric_name = "AUC"
        else:
            final_metric = accuracy_score(y_test_full, y_pred)
            metric_name = "Accuracy"
    else:
        from sklearn.metrics import mean_squared_error
        final_metric = mean_squared_error(y_test_full, y_pred)
        metric_name = "MSE"

    print(f"\n--- Final Model Evaluation ---")
    print(f"Metric ({metric_name}): {final_metric:.4f}")
    print(f"Training time for final model: {fit_time:.2f} seconds")

    return model, final_metric

X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(
    X_raw, y_encoded, test_size=0.2, random_state=RANDOM_STATE,
    stratify=y_encoded
)

if study.best_trial:
    best_params = study.best_trial.params
    final_model, final_score = evaluate_final_model(
        best_params, X_train_full, y_train_full, X_test_full, y_test_full,
        task_type=TASK_TYPE, preprocessor=data_preprocessor
    )

    joblib.dump(final_model, 'best_rf_blood_transfusion_model_fixed.pkl')
    joblib.dump(data_preprocessor, 'data_preprocessor_blood_transfusion_fixed.pkl')
    print("Best model and preprocessor for Blood Transfusion saved (fixed pruner run).")

Loaded datasets/php0iVrYT.arff. Shape: (748, 5)
Target column: 'Class'
Numerical features: ['V1', 'V2', 'V3', 'V4']
Categorical features detected: []
Encoded target classes: ['1' '2']
Starting Optuna study with 100 trials for Blood Transfusion dataset (FIXED PRUNER)...


  0%|          | 0/100 [00:00<?, ?it/s]

Trial 1, Step 0: 2 trees, OOB: 0.7040

=== DEBUGGING TRIAL 1 ===
Intermediate values: {0: 0.7040133779264214}
Current step: 0, Current value: 0.7040133779264214
[Debug Trial 1]: Not enough steps (1 < 3)
Trial 1, Step 1: 6 trees, OOB: 0.6973

=== DEBUGGING TRIAL 1 ===
Intermediate values: {0: 0.7040133779264214, 1: 0.697324414715719}
Current step: 1, Current value: 0.697324414715719
[Debug Trial 1]: Not enough steps (2 < 3)
Trial 1, Step 2: 12 trees, OOB: 0.6973

=== DEBUGGING TRIAL 1 ===
Intermediate values: {0: 0.7040133779264214, 1: 0.697324414715719, 2: 0.697324414715719}
Current step: 2, Current value: 0.697324414715719
Steps for prediction: [0 1 2]
Values for prediction: [0.70401338 0.69732441 0.69732441]
[Debug Trial 1]: Curve fit params: a=26.8916, b=26.1887, c=-0.0001
[Debug Trial 1]: Extrapolated to step 6: 0.6828
Predicted final performance: 0.6828270728005563
[Debug Trial 1]: No completed trials yet
Trial 1, Step 3: 25 trees, OOB: 0.7057

=== DEBUGGING TRIAL 1 ===
Intermedia

C:\Users\Obidur Rahman\AppData\Local\Temp\ipykernel_3964\3441492807.py:144: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(saturating_func, steps, values, p0=p0, maxfev=1000)


Trial 1, Step 4: 62 trees, OOB: 0.7107

=== DEBUGGING TRIAL 1 ===
Intermediate values: {0: 0.7040133779264214, 1: 0.697324414715719, 2: 0.697324414715719, 3: 0.705685618729097, 4: 0.7107023411371237}
Current step: 4, Current value: 0.7107023411371237
Steps for prediction: [0 1 2 3 4]
Values for prediction: [0.70401338 0.69732441 0.69732441 0.70568562 0.71070234]
[Debug Trial 1]: Curve fit params: a=44.9377, b=44.2390, c=0.0000
[Debug Trial 1]: Extrapolated to step 12: 0.7247
Predicted final performance: 0.7247434634773597
[Debug Trial 1]: No completed trials yet
Trial 1, Step 5: 125 trees, OOB: 0.7124

=== DEBUGGING TRIAL 1 ===
Intermediate values: {0: 0.7040133779264214, 1: 0.697324414715719, 2: 0.697324414715719, 3: 0.705685618729097, 4: 0.7107023411371237, 5: 0.7123745819397993}
Current step: 5, Current value: 0.7123745819397993
Steps for prediction: [0 1 2 3 4 5]
Values for prediction: [0.70401338 0.69732441 0.69732441 0.70568562 0.71070234 0.71237458]
[Debug Trial 1]: Curve fittin

...

[Debug Trial 91]: Curve fit params: a=99.9992, b=99.2539, c=0.0001
[Debug Trial 91]: Extrapolated to step 6: 0.8054
Predicted final performance: 0.8054487248557933
Completed trial values: [0.7905701754385964, 0.7938596491228068, 0.797027290448343, 0.7860623781676414, 0.7928849902534112, 0.7807017543859648, 0.790204678362573, 0.7897173489278753, 0.7887426900584794, 0.7897173489278753, 0.7902046783625731, 0.7877680311890839, 0.7958089668615984, 0.7926413255360623, 0.7923976608187134, 0.7928849902534112, 0.7941033138401559, 0.7921539961013644, 0.7923976608187134, 0.7926413255360623, 0.7923976608187133, 0.7938596491228069, 0.7945906432748538, 0.7938596491228069, 0.7945906432748537, 0.7950779727095516, 0.7945906432748536, 0.7945906432748536, 0.7948343079922026, 0.7945906432748537, 0.7943469785575048, 0.794103313840156, 0.7921539961013645]
Threshold (quantile 0.5): 0.7928849902534112
Prediction vs threshold: 0.8054 vs 0.7929
Decision: CONTINUE
✅ CONTINUING: 0.8054 >= 0.7929
Trial 91, Step 3:

C:\Users\Obidur Rahman\AppData\Local\Temp\ipykernel_3964\3441492807.py:144: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(saturating_func, steps, values, p0=p0, maxfev=1000)


Trial 92, Step 5: 111 trees, OOB: 0.7910

=== DEBUGGING TRIAL 92 ===
Intermediate values: {0: 0.7408026755852842, 1: 0.7491638795986622, 2: 0.7759197324414716, 3: 0.7892976588628763, 4: 0.7892976588628763, 5: 0.7909698996655519}
Current step: 5, Current value: 0.7909698996655519
Steps for prediction: [0 1 2 3 4 5]
Values for prediction: [0.74080268 0.74916388 0.77591973 0.78929766 0.78929766 0.7909699 ]
[Debug Trial 92]: Curve fit params: a=0.8048, b=0.0673, c=0.3734
[Debug Trial 92]: Extrapolated to step 15: 0.8045
Predicted final performance: 0.8045095067861355
Completed trial values: [0.7905701754385964, 0.7938596491228068, 0.797027290448343, 0.7860623781676414, 0.7928849902534112, 0.7807017543859648, 0.790204678362573, 0.7897173489278753, 0.7887426900584794, 0.7897173489278753, 0.7902046783625731, 0.7877680311890839, 0.7958089668615984, 0.7926413255360623, 0.7923976608187134, 0.7928849902534112, 0.7941033138401559, 0.7921539961013644, 0.7923976608187134, 0.7926413255360623, 0.79239

C:\Users\Obidur Rahman\AppData\Local\Temp\ipykernel_3964\3441492807.py:144: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(saturating_func, steps, values, p0=p0, maxfev=1000)


Trial 98, Step 3: 27 trees, OOB: 0.7960

=== DEBUGGING TRIAL 98 ===
Intermediate values: {0: 0.7408026755852842, 1: 0.7575250836120402, 2: 0.7842809364548495, 3: 0.7959866220735786}
Current step: 3, Current value: 0.7959866220735786
Steps for prediction: [0 1 2 3]
Values for prediction: [0.74080268 0.75752508 0.78428094 0.79598662]
[Debug Trial 98]: Curve fit params: a=0.9369, b=0.1972, c=0.1154
[Debug Trial 98]: Extrapolated to step 9: 0.8671
Predicted final performance: 0.867119242147773
Completed trial values: [0.7905701754385964, 0.7938596491228068, 0.797027290448343, 0.7860623781676414, 0.7928849902534112, 0.7807017543859648, 0.790204678362573, 0.7897173489278753, 0.7887426900584794, 0.7897173489278753, 0.7902046783625731, 0.7877680311890839, 0.7958089668615984, 0.7926413255360623, 0.7923976608187134, 0.7928849902534112, 0.7941033138401559, 0.7921539961013644, 0.7923976608187134, 0.7926413255360623, 0.7923976608187133, 0.7938596491228069, 0.7945906432748538, 0.7938596491228069, 0.

In [3]:
# --- Optional: Optuna Visualizations ---
# To use these, you need to install plotly and kaleido: pip install plotly kaleido
import optuna.visualization as ov
fig1 = ov.plot_optimization_history(study)
fig1.show()
fig2 = ov.plot_intermediate_values(study)
fig2.show()
fig3 = ov.plot_param_importances(study)
fig3.show()